In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, ImageMessage, TextSendMessage
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")
app_1 = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )
# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "素材/問題一/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = {
      "type": "text",
      "text": "哈囉!" + str(user_profile.display_name) + "\n歡迎您使用「履視步爽」"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
    
#     #個人message.json
#     ID_user = user_profile.display_name
#     print(ID_user)
#     ID_data = {
#         "user_ID" : user_profile.display_name,
#         "folder" : [],
#         "menu" : []
#     }
#     ID_name = str(user_profile.display_name)+".json"
#     dn = "ID_message/"
#     if not os.path.exists(dn):
#         os.makedirs(dn)
#     f = open(dn+ID_name, "w", encoding="utf-8")
#     json.dump(ID_data, f)
#     f.close()
        
    #綁定 rich_menu
    linkRichMenuId = open("素材/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)


    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [5]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

無則發送訊息"已記下，放在心上，未來精準推送"

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)
from os import listdir

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    
    # 指定要列出素材的目錄
    mypath = "素材/"
    # 取得所有檔案與子目錄名稱
    list_text = listdir(mypath)
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]

    if event.message.text in list_text:
        
        replyJsonPath = "素材/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    else:
        pass
#         hello = {
#           "type": "text",
#           "text": "已記下，放在心上，開始推送"
#         }
#         result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))

#         # 發送
#         line_bot_api.reply_message(
#             event.reply_token,
#             result_message_array
#         )


In [6]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送
    擷取tag訊息

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送
    擷取tag訊息

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 
import time

@handler.add(PostbackEvent)
def process_postback_event(event):
    

    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        #讀取其reply.json，轉譯成消息
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n') 
        top_line = {
          "type": "text",
          "text": "已記下，放在心上，開始推送"
        }
        result_message_array.insert(0, TextSendMessage.new_from_json_dict(top_line))
  
        ##發送訊息
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
            
        )
    elif 'menu' in query_string_dict:
    
        #讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        #讀取其reply.json，轉譯成消息
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n') 
        top_line = {
          "type": "text",
          "text": "已記下，放在心上，開始推送"
        }
        result_message_array.insert(0, TextSendMessage.new_from_json_dict(top_line))
  
        #發送訊息
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        ) 

In [7]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取照片的主顏色

'''
#!pip install clarifai
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
        
    message_content = line_bot_api.get_message_content(event.message.id)
    
    #擷取照片的主顏色
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    #路徑為相對路徑
    Image_shoes = './images/'+event.message.id+'.jpg'
    # ======================= 官方固定格式 ===============================
    app = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=Image_shoes)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
    print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
        print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
    print(color_datas)
    print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)

    #同時存在一個，上限五則訊息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id+"\n"+color_data))


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.2 - - [17/Oct/2019 06:22:07] "POST / HTTP/1.1" 200 -


{'folder': ['問題六'], 'tag': ['populartype']}


172.18.0.2 - - [17/Oct/2019 06:22:11] "POST / HTTP/1.1" 200 -
172.18.0.2 - - [17/Oct/2019 06:22:19] "POST / HTTP/1.1" 200 -


{'folder': ['rich_menu_4'], 'tag': ['america']}


172.18.0.2 - - [17/Oct/2019 06:22:21] "POST / HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

## TEST

### 擷取tag值

In [ ]:
# h={"data": "folder=問題二&tag=woman"}
# dn = "ID_message/"+ "harry hung.json"
# with open(dn, 'r') as f:
#     print(f)
#     data = json.load(f)
#     print(data)
#     print(data["folder"])
#     print(h)
#     data["folder"].append(h["data"])
#     print(data)
#     data_1 =data
# with open(dn, 'w') as f:
#     json.dump(data_1, f)
#     print(data_1)

### 從tag取值並輸出輪播

In [ ]:
# #!pip install pandas
# import json
# import pandas as pd

# #接收line的輸出值:資料或照片

# #把資料輸出至程式中並輸出結果
# dc = "ID_message/"+ "harry hung.json"
# list_tag = []
# with open(dc, 'r') as f:
#     data = json.load(f)
#     list_tag = data["folder"]
#     print(list_tag)

# # #把輸出結果的資料備一份丟給推薦系統(使用習慣)
# # with open(dc, 'r') as f:
# #     print(f)
# #     data = json.load(f)
# #     print(data)
# #     print(data["folder"])
# #     print(query_string_dict.get('tag')[0])
# #     data["folder"].append(query_string_dict.get('tag')[0])
# #     print(data)
# #     data_1 =data
# # with open(dc, 'w') as f:
# #     json.dump(data_1, f)
# #     print(data_1)
# #連接到目的地:資料庫查詢&輸出
# dn = "./database/adidas.csv"
# list_tag[0:6]
# catory=["Basketball", "Running", "Lifestyle"]
# df = pd.read_csv(dn, encoding = "utf-8")
# Data_tag = {
#     "man" : ["男女款", "U", "M"],
#     "woman" : ["男女款", "U", "W"],
#     "3000" : ["價格", "< 3000"],
#     "3000~5000" : ["價格", "> 3000", "< 5000"],
#     "5000" : ["價格", "> 5000"],
#     "run" : ["品牌","Running"],
#     "basketball" : ["品牌","Basketball"],
#     "sport" : ["品牌","not in catory"],
#     "other" : ["品牌","Lifestyle"],
#     "blackwhite" : ["顏色", "white", "black"],
#     "onlywhite" : ["顏色", "white"],
#     "onlycolor" : ["顏色", "len(color=[])=1"],
#     "morecolor" : ["顏色", "len(color=[])>1"],
#     "populartype" : ["商品介紹",""],
#     "classictype" : ["商品介紹",""],
#     "alltype" : ["商品介紹",""],
#     "america" : ["品牌",""],
#     "japan" : ["品牌",""]
# }
# for i in list_tag:
#     col_1 = df[Data_tag[i][0]]
    
#     if len(Data_tag[i]) == 2:
#         name=df[col_1].str.contains[Data_tag[i][1]]
#     else:
#         name=df[col_1].str.contains[Data_tag[i][1]]
#         name_1=df[col_1].str.contains[Data_tag[i][2]]
#         name_2 = name + name_1
        

#準備空的list
# list_col_color = []

#     #到目的地取值
#     web_pic = list_tag
#     main_color =
#     second_color =
#     web_shoes =
#     name_shoes = 

#     #循環加到 list_col_color
#     col_1 = {
#             "thumbnailImageUrl": web_pic
#             "title": "所顏是否?",
#             "text": main_color,
#             "actions": [
#               {
#                 "type": "postback",
#                 "label": second_color,
#                 "text": web_shoes,
#                 "data": "tag=maincolor|Secondarycolor|"+name_shoes
#               }
#             ]
#           }
#     list.append(col_1)

#     #輸出到linebot輪播
#     dict={
#           "type": "template",
#           "altText": "this is a carousel template",
#           "template": {
#             "type": "carousel",
#             "actions": [],
#             "columns":  list_col_color
#               }
#           }
#     json=[dict]
